In [1]:
import numpy as np
import scipy.io as sio
import matplotlib.pyplot as plt


fname = '../data/100_unrelated.csv'
subjectids = np.loadtxt(fname, dtype=np.int)
nSubj = len(subjectids)
tasks = ['rfMRI_REST1_LR', 'rfMRI_REST1_RL', 'rfMRI_REST2_LR',
         'rfMRI_REST2_RL', 'tfMRI_EMOTION_LR', 'tfMRI_EMOTION_RL',
         'tfMRI_GAMBLING_LR', 'tfMRI_GAMBLING_RL', 'tfMRI_LANGUAGE_LR',
         'tfMRI_LANGUAGE_RL', 'tfMRI_MOTOR_LR', 'tfMRI_MOTOR_RL',
         'tfMRI_RELATIONAL_LR', 'tfMRI_RELATIONAL_RL', 'tfMRI_SOCIAL_LR',
         'tfMRI_SOCIAL_RL', 'tfMRI_WM_LR', 'tfMRI_WM_RL']

M = {}
for task in tasks:
    masterFC_dir = '../data/results_SIFT2'
    restingstatename = 'fMRI/' + task + '/FC/FC_glasser_subc_GS_bp_z.mat'
    task_matrices = []
    for subject in subjectids:
        filename = masterFC_dir + '/' + str(subject) + '/' + restingstatename
        mat = sio.loadmat(filename, squeeze_me=True, struct_as_record=False)
        A_orig = mat['FC']
        A_orig = (A_orig + A_orig.T) / 2
        np.fill_diagonal(A_orig, 0)
        task_matrices.append(A_orig)
    # This is the 100 by 374 by 374 matrices of 100 unrelated subjects
    task_matrices = np.array(task_matrices).reshape(len(subjectids),-1)
    M[task] = task_matrices


In [2]:
task_list = ["Rest", "Emotion", "Gambling", "Language", "Motor", "Relational", "Social", "WM"]

In [3]:
all_FC = np.hstack((M['rfMRI_REST1_LR'],M['rfMRI_REST1_RL'],M['tfMRI_EMOTION_LR'],M['tfMRI_EMOTION_RL'],
                  M['tfMRI_GAMBLING_LR'], M['tfMRI_GAMBLING_RL'], M['tfMRI_LANGUAGE_LR'], M['tfMRI_LANGUAGE_RL'],
                  M['tfMRI_MOTOR_LR'], M['tfMRI_MOTOR_RL'], M['tfMRI_RELATIONAL_LR'], M['tfMRI_RELATIONAL_RL'],
                  M['tfMRI_SOCIAL_LR'], M['tfMRI_SOCIAL_RL'], M['tfMRI_WM_LR'], M['tfMRI_WM_RL'] )).reshape(-1,M['tfMRI_WM_LR'].shape[1])

## Deep Learning

In [4]:
import torch
from torch import nn, optim
from torch.utils import data
from sklearn.preprocessing import StandardScaler

# GPU is available? If so, we use it.
use_cuda = torch.cuda.is_available() 
device = torch.device("cuda:0" if use_cuda else "cpu")
if use_cuda:
    print("GPU detected. Will use GPU for training!")
    torch.backends.cudnn.benchmark = True
else:
    print("No GPU detected. Will use CPU for training.")

GPU detected. Will use GPU for training!


In [5]:
labels = torch.tensor(np.repeat(np.arange(0,8),nSubj*2), dtype=torch.long)
indices = np.random.permutation(labels.shape[0])
train_idx = indices[:int(0.6*labels.shape[0])]
val_idx = indices[int(0.6*labels.shape[0]):int(0.8*labels.shape[0])]
test_idx = indices[int(0.8*labels.shape[0]):]

In [6]:
std_scale = StandardScaler().fit(all_FC[train_idx,:])

In [7]:
all_FC[train_idx,:] = std_scale.transform(all_FC[train_idx,:])
all_FC[val_idx,:] = std_scale.transform(all_FC[val_idx,:])
all_FC[test_idx,:] = std_scale.transform(all_FC[test_idx,:])

In [8]:
all_FC.shape

(1520, 139876)

In [9]:
all_FC = torch.FloatTensor(all_FC)

In [10]:
train_data = []
for i in train_idx:
    train_data.append([all_FC[i], labels[i]])
    
val_data = []
for i in val_idx:
    val_data.append([all_FC[i], labels[i]])

test_data = []
for i in test_idx:
    test_data.append([all_FC[i], labels[i]])

In [11]:
train_loader = torch.utils.data.DataLoader(train_data, shuffle=True, batch_size=3)
val_loader = torch.utils.data.DataLoader(val_data, shuffle=True, batch_size=3)
test_loader = torch.utils.data.DataLoader(test_data, shuffle=True, batch_size=3)

## NN Architecture

In [12]:
input_size = all_FC.shape[1] # number of columns
hidden_sizes = [2048, 128]
output_size = 8
max_epochs = 30
n_epochs_stop = 5

In [16]:
model = nn.Sequential(nn.Linear(input_size, hidden_sizes[0]),
                      nn.BatchNorm1d(hidden_sizes[0]),
                      nn.ReLU(),
                      nn.Dropout(),
                      nn.Linear(hidden_sizes[0], hidden_sizes[1]),
                      nn.BatchNorm1d(hidden_sizes[1]),
                      nn.ReLU(),
                      nn.Linear(hidden_sizes[1], output_size))
if use_cuda:
    model = model.cuda()

RuntimeError: CUDA out of memory. Tried to allocate 1.07 GiB (GPU 0; 2.00 GiB total capacity; 1.07 GiB already allocated; 169.50 MiB free; 1.09 GiB reserved in total by PyTorch)

In [14]:
loss_fn = nn.CrossEntropyLoss()
opt = optim.SGD(model.parameters(), lr=0.01)
print(model)

Sequential(
  (0): Linear(in_features=139876, out_features=2048, bias=True)
  (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (2): BatchNorm1d(2048, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (3): ReLU()
  (4): Dropout(p=0.5, inplace=False)
  (5): Linear(in_features=2048, out_features=128, bias=True)
  (6): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (7): ReLU()
  (8): Linear(in_features=128, out_features=8, bias=True)
)


In [15]:

history = {}
history['loss'] = []
history['val_loss'] = []
history['acc'] = []
history['val_acc'] = []
min_val_loss = np.Inf
early_stop = False
# Loop over epochs


for epoch in range(max_epochs):
    # Training
    model.train()
    train_loss         = 0.0
    num_train_correct  = 0
    num_train_examples = 0
    for local_batch, local_labels in train_loader:
        # Transfer to GPU
        if use_cuda:
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        
        

        opt.zero_grad()
        output = model(local_batch)
        loss = loss_fn(output, local_labels)
        loss.backward()
        opt.step()
        
        train_loss += loss.data.item() * local_batch.size(0)
        num_train_correct  += (torch.max(output, 1)[1] == local_labels).sum().item()
        num_train_examples += local_batch.shape[0]
        
        train_acc = num_train_correct / num_train_examples
        train_loss  = train_loss / len(train_loader.dataset)
        
    # Validation
    model.eval()
    val_loss       = 0.0
    num_val_correct  = 0
    num_val_examples = 0
    with torch.set_grad_enabled(False):
        for local_batch, local_labels in val_loader:
            # Transfer to GPU
            if use_cuda:
                local_batch, local_labels = local_batch.to(device), local_labels.to(device)
            output = model(local_batch)
            loss = loss_fn(output, local_labels)
            
            val_loss += loss.data.item() * local_batch.size(0)
            num_val_correct  += (torch.max(output, 1)[1] == local_labels).sum().item()
            num_val_examples += local_batch.shape[0]
            
        val_acc  = num_val_correct / num_val_examples
        val_loss = val_loss / len(val_loader.dataset)
        
        if val_loss < min_val_loss:
            epochs_no_improve = 0
            min_val_loss = val_loss
        else:
            epochs_no_improve += 1
  # Check early stopping condition
        if epochs_no_improve == n_epochs_stop:
            early_stop = print('Early stopping!')
            break

        
    print(f'Epoch {epoch+1}/{max_epochs}, train loss: {train_loss:5.4f}, train acc: {train_acc:5.3f}, val loss: {val_loss:5.3f}, val acc: {val_acc:5.3f}')
    
    
    history['loss'].append(train_loss)
    history['val_loss'].append(val_loss)
    history['acc'].append(train_acc)
    history['val_acc'].append(val_acc)
    
    if early_stop:
        print("Stopped")
        break

RuntimeError: non-empty 3D or 4D (batch mode) tensor expected for input

In [ ]:
local_batch.device

In [ ]:
model.eval()
num_correct  = 0
num_examples = 0
with torch.set_grad_enabled(False):
    for local_batch, local_labels in test_loader:
        # Transfer to GPU
        if use_cuda:
            local_batch, local_labels = local_batch.to(device), local_labels.to(device)
        output = model(local_batch)
        num_correct  += (torch.max(output, 1)[1] == local_labels).sum().item()
        num_examples += local_batch.shape[0]
            
    test_acc  = num_correct / num_examples

In [ ]:
plt.plot(history['loss'])
plt.plot(history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
plt.plot(history['acc'])
plt.plot(history['val_acc'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Validation'], loc='upper left')
plt.show()

In [ ]:
test_acc